In [1]:
"""
    构造数据:计算vintage指标需要【放款月、放款金额、观测月(还款日期)、应还本金、实还本金、贷款状态】
"""
import pandas as pd
from faker import Faker
from datetime import timedelta
from random import randint
import datetime
fake = Faker('zh_CN')


# 放款表
loans = []
for i in range(20):
    days = randint(1, 180)
    date = datetime.date.today() - timedelta(days=days)
    amount = randint(1000, 10000)
    id = i + 1
    loans.append({'date': date, 'amount': amount, 'id': id})

loans_df = pd.DataFrame(loans).sort_values('date')

# 还款表
repayments = []  
for i in range(20):
    days = randint(1, 180)
    due_date = datetime.date.today() - timedelta(days=days)
    paid_date = due_date + timedelta(days=randint(1,5))
    due_principal = randint(500, 5000)
    paid_principal = randint(500, 5000)
    repayments.append({'due_date': due_date, 'paid_date': paid_date,
                       'due_principal': due_principal, 'paid_principal': paid_principal})
                       
repayments_df = pd.DataFrame(repayments).sort_values('due_date')

In [2]:
# 计算每月的放款本金
concat_df = pd.concat([loans_df,repayments_df], axis=1)
concat_df = concat_df[['date','due_date','paid_date','due_principal','amount','paid_principal','id']]
table1 = concat_df.groupby('due_date',as_index=False).agg({'due_principal':sum})
dueDate_duePrincipal_Map = table1.set_index('due_date').to_dict()['due_principal']
concat_df['SUM_duePrincipal'] = concat_df['due_date'].map(dueDate_duePrincipal_Map)

concat_df = concat_df.rename(columns = {"date":"放款日期",
                                        "due_date":"应还款日期",
                                        "paid_date":"实际还款日期",
                                        "amount":"放款金额",
                                        "id":"放款编号",
                                        "due_principal":"应还本金",
                                        "paid_principal":"实还本金",
                                        "SUM_duePrincipal":"总应还金额"}
                            )

concat_df = concat_df.reset_index(drop=True)

In [104]:
# 人为设定观察月份
from dateutil.relativedelta import relativedelta

concat_df["观察月份"] = concat_df["放款日期"] + relativedelta(months=1)
concat_df["观察月份"] = pd.to_datetime(concat_df["观察月份"])
concat_df["观察月份"] = concat_df["观察月份"].dt.strftime("%Y-%m")
concat_df["放款月份"] = pd.to_datetime(concat_df["放款日期"]).dt.strftime("%Y-%m") # 修改放款月格式

# 计算账龄MOB值
concat_df["MOB"] = concat_df.apply(lambda x:
                                   int(x["观察月份"][-2:]) - x["放款日期"].month if int(x["观察月份"][:4])==x["放款日期"].year else
                                   int(x["观察月份"][-2:]) - x["放款日期"].month + 12 if int(x["观察月份"][:4])==x["放款日期"].year+1 else 0,
                                   axis=1
                                  )

# 计算剩余本金
concat_df["剩余本金"] = concat_df["应还本金"] - concat_df["实还本金"]

# 计算MOB_df
MOB_df = concat_df.groupby(["放款月份","观察月份","MOB"],as_index=False)[["实还本金","应还本金","剩余本金","放款金额"]] \
                  .agg({"实还本金":"sum", "应还本金":"sum", "剩余本金":"sum", "放款金额":"sum"})

# 计算剩余本金占放款金额比例的vintage
MOB_df["mob_1"] = MOB_df["剩余本金"] / MOB_df["放款金额"]

In [110]:
MOB_df[["放款月份","观察月份","mob_1"]]

,放款月份,观察月份,mob_1
0,2023-02,2023-03,0.207423
1,2023-03,2023-04,0.550189
2,2023-04,2023-05,-0.289719
3,2023-05,2023-06,-0.142036
4,2023-06,2023-07,0.105153
5,2023-07,2023-08,-0.055983
